# Extending MUSE

One key feature of the generalized sector's implementation is that it should be easy to
extend. As such, MUSE can be made to run custom python functions,
as long as these inputs and output of the function follow a standard specific to each
step. We will look at a few here. 

Below is a list of possible hooks, referenced by their
implementation in the MUSE model:

- `register_interaction_net` in `muse.interactions`: a list of lists of agents
  that interact together.
- `register_agent_interaction` in `muse.interactions`: Given a list of
  interacting agents, perform the interaction.
- `register_production` in `muse.production`: A method to compute the production
  from a sector, given the demand and the capacity.
- `register_initial_asset_transform` in `muse.hooks`: Allows any kind of transformation to be applied to the assets of an agent, prior to investing.
- `register_final_asset_transform` in `muse.hooks`: After computing the investment, this sets the assets that will be owned by the agents.
- `register_demand_share` in `muse.demand_share`: During agent investment, this is the share
  of the demand that an agent will try and satisfy.
- `register_filter` in `muse.filters`:   A filter to remove technologies from consideration, during agent investment. 
- `register_objective` in `muse.objectives`: A quantity which allows an agent to compare technologies during investment.
- `register_decision` in `muse.decisions`: A transformation applied to aggregate multiple objectives into a single objective during agent investment, e.g. via a weighted sum.
- `register_investment` in `muse.investment`: During agent investment, matches
  the demand for future investment using the decision metric above.
- `register_output_quantity` in `muse.output.sector`: A sectorial quantity to output for
  postmortem analysis.
- `register_output_sink` in `muse.outputs`: A _place_ to store an output
  quantity, e.g. a file with a given format, a database on premise or on the cloud,
  etc...
- `register_carbon_budget_fitter` in `muse.carbon_budget`
- `register_carbon_budget_method` in `muse.carbon_budget`
- `register_sector`: Registers a function that can create a sector from a muse
  configuration object.

## Extending outputs

MUSE can be used to save custom quantities as well as data for analysis. There are two steps to this process:
    
- Computing the quantity of interest
   
- Store the quantity of interest in a sink

In practice, this means that we can compute any quantity, such as capacity or consumption of an energy source and save it to a csv file, or a netcdf file.

### Output extension

To demonstrate this, we will compute a new edited quantity of consumption, then save it as a text file.

The current implementation of the quantity of consumption found in `muse.outputs.sector` filters out values of 0. In this example, we would like to maintain the values of 0, but do not want to edit the source code of MUSE.

This is rather simple to do using MUSE's hooks.

First we create a new function called `consumption_zero` as follows:

In [1]:
from muse.outputs import register_output_quantity
from muse.outputs.sector import market_quantity
from xarray import Dataset, DataArray
from typing import Optional, List, Text

@register_output_quantity
def consumption_zero(
    market: Dataset,
    capacity: DataArray,
    technologies: Dataset,
):
    """Current consumption."""
    result = (
        market_quantity(market.consumption, sum_over="timeslice", drop=None)
        .rename("consumption")
        .to_dataframe()
        .round(4)
    )
    return result

The function we created takes three arguments. These arguments (`market`, `capacity` and `technology`) are mandatory for the `@register_output_quantity` hook. Other hooks require different arguments. 

Whilst this function is very similar to the `consumption` function in `muse.outputs.sector`, we have modified it slightly by allowing for values of `0`.

The important part of this function is the `@register_output_quantity` decorator. This decorator ensures that this new quantity is addressable in the TOML file. Notice that we did not need to edit the source code to create our new function.

Next, we can create a sink to save the output quantity previously registered. For this example, this sink will simply dump the quantity it is given to a file, with the "Hello world!" message:

In [2]:
from typing import Any, Text
from muse.outputs.sinks import register_output_sink, sink_to_file

@register_output_sink(name="txt")
@sink_to_file(".txt")
def text_dump(data: Any, filename: Text) -> None:
    from pathlib import Path
    Path(filename).write_text(f"Hello world!\n\n{data}")

The code above makes use of two dectorators: `@register_output_sink` and `@sink_to_file`. 

`@register_output_sink` registers the function with MUSE, so that the sink is addressable from a TOML file. The second one, `@sink_to_file`, is optional. This adds some nice-to-have features to sinks that are files. For example, a way to specify filenames and check that files cannot be overwritten, unless explicitly allowed to.

Next, we want to modify the TOML file to actually use this output type. To do this, we add a section to the output table:

```toml
[[sectors.residential.outputs]]
quantity = "consumption_zero"
sink = "txt"
filename = "{cwd}/{default_output_dir}/{Sector}{Quantity}{year}{suffix}"
```

The last line above allows us to specify the name of the file. We could also use `sector` above or `quantity`.

There can be as many sections of this kind as we like in the TOML file, which allow for multiple outputs.

Next, we  first copy the default model provided with muse to a local subfolder called "model". Then we read the `settings.toml` file and modify it using python. You may prefer to modify the `settings.toml` file using your favorite text editor. However, modifying the file programmatically allows us to
routinely run this notebook as part of MUSE's test suite and check that the tutorial it is still up
to date.

In [3]:
from pathlib import Path
from toml import load, dump
from muse import examples

model_path = examples.copy_model(overwrite=True)
settings = load(model_path / "settings.toml")
new_output = {
    "quantity": "consumption_zero",
    "sink":  "txt",
    "overwrite": True,
    "filename": "{cwd}/{default_output_dir}/{Sector}{Quantity}{year}{suffix}",
}
settings["sectors"]["residential"]["outputs"].append(new_output)
dump(settings, (model_path / "modified_settings.toml").open("w"))
settings

-- 2020-11-09 11:19:48 - muse.sectors.register - INFO
Sector legacy registered.

-- 2020-11-09 11:19:48 - muse.sectors.register - INFO
Sector preset registered, with alias presets.

-- 2020-11-09 11:19:48 - muse.sectors.register - INFO
Sector default registered.



{'time_framework': [2020, 2025, 2030, 2035, 2040, 2045, 2050],
 'foresight': 5,
 'regions': ['R1'],
 'interest_rate': 0.1,
 'interpolation_mode': 'Active',
 'log_level': 'info',
 'equilibrium_variable': 'demand',
 'maximum_iterations': 100,
 'tolerance': 0.1,
 'tolerance_unmet_demand': -0.1,
 'outputs': [{'quantity': 'prices',
   'sink': 'aggregate',
   'filename': '{cwd}/{default_output_dir}/MCA{Quantity}.csv'},
  {'quantity': 'capacity',
   'sink': 'aggregate',
   'filename': '{cwd}/{default_output_dir}/MCA{Quantity}.csv'}],
 'carbon_budget_control': {'budget': []},
 'global_input_files': {'projections': '{path}/input/Projections.csv',
  'global_commodities': '{path}/input/GlobalCommodities.csv'},
 'sectors': {'residential': {'type': 'default',
   'priority': 1,
   'dispatch_production': 'share',
   'technodata': '{path}/technodata/residential/Technodata.csv',
   'commodities_in': '{path}/technodata/residential/CommIn.csv',
   'commodities_out': '{path}/technodata/residential/CommOut

We can now run the simulation. There are two ways to do this. From the command-line, where we can do:

    python3 -m muse data/commercial/modified_settings.toml 

(note that slashes may be the other way on Windows). Or directly from the notebook:

In [4]:
import logging
from muse.mca import MCA
logging.getLogger("muse").setLevel(0)
mca = MCA.factory(model_path / "modified_settings.toml")
mca.run();

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 148.9679256735      
0.2598249156018     0.2598249156018     0.2598249156018     0.7495200004432  0.2598249156018     120.5733849622      
0.02399956829695    0.02399956829695    0.02399956829695    0.9210391224498  0.02399956829695    4.780663765494      
0.0181364461758     0.0181364461758     0.0181364461758     0.2509588065043  0.0181364461758     7.107141691547      
0.01499350833129    0.01499350833129    0.01499350833129    0.1921973185437  0.01499350833129    70.77614035582      
0.004968295711366   0.004968295711367   0.004968295711366   0.6857131120066  0.004968295711366   164.7472224003      
0.0006443120819652  0.0006443120819642  0.000644312081964   0.8804718592549  0.0006443120819672  289.7109372802      
2.427431365313e-06  2.427431365276e-06  2.42743136527e-06

-- 2020-11-09 11:19:59 - muse.mca - WARNING
Check growth constraints for wind.



Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 359.2443189825      
0.2131118149695     0.2131118149695     0.2131118149695     0.7960343319409  0.2131118149695     262.2885392799      
0.05758094728718    0.05758094728718    0.05758094728718    0.7523513573813  0.05758094728718    13.16175379893      
0.01048349585899    0.01048349585899    0.01048349585899    0.8203940076989  0.01048349585899    9.036399448741      
0.009005598049604   0.009005598049604   0.009005598049604   0.1488155054953  0.009005598049604   19.40296370843      
0.002317604003518   0.002317604003518   0.002317604003518   0.8098673571979  0.002317604003518   226.5492459765      
0.0009784310820962  0.0009784310820963  0.0009784310820963  0.5991550275447  0.0009784310820971  289.4684048776      
0.0001326875071986  0.0001326875071986  0.000132687507198

-- 2020-11-09 11:20:09 - muse.mca - WARNING
Check growth constraints for wind.



Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 359.2443189825      
0.2131118149695     0.2131118149695     0.2131118149695     0.7960343319409  0.2131118149695     262.2885392799      
0.05758094728718    0.05758094728718    0.05758094728718    0.7523513573813  0.05758094728718    13.16175379893      
0.01048349585899    0.01048349585899    0.01048349585899    0.8203940076989  0.01048349585899    9.036399448741      
0.009005598049604   0.009005598049604   0.009005598049604   0.1488155054953  0.009005598049604   19.40296370843      
0.002317604003518   0.002317604003518   0.002317604003518   0.8098673571979  0.002317604003518   226.5492459765      
0.0009784310820962  0.0009784310820963  0.0009784310820963  0.5991550275447  0.0009784310820971  289.4684048776      
0.0001326875071986  0.0001326875071986  0.000132687507198

...

/Users/alexkell/Documents/SGI/1-examples/example_model/model/Results/muse/src/muse/investments.py:325: OptimizeWarning: Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.
  res = linprog(**adapter.kwargs, options=dict(disp=True))
/Users/alexkell/Documents/SGI/1-examples/example_model/model/Results/muse/src/muse/investments.py:325: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
  res = linprog(**adapter.kwargs, options=dict(disp=True))
/Users/alexkell/anaconda3/lib/python3.8/site-packages/scipy/optimize/_linprog_ip.py:116: LinAlgWarning: Ill-conditioned matrix (rcond=1.62544e-36): result may not be accurate.
  return sp.linalg.solve(M

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 647.4402248427      
0.1042702416317     0.1042702416317     0.1042702416317     0.9044651903652  0.1042702416317     197.7436068274      
0.04330783922975    0.04330783922975    0.04330783922975    0.6213338928986  0.04330783922975    554.8458695929      
0.02029339907042    0.02029339907042    0.02029339907042    0.5546164251713  0.02029339907042    887.503418141       
0.002059630009005   0.002059630008646   0.002059630008646   0.9123945707354  0.002059630008855   1773.497308596      
1.96164830749e-06   1.961648306624e-06  1.961648306623e-06  0.9995475315428  1.96164830693e-06   1899.795955905      
1.183904657193e-10  1.183907029537e-10  1.183907180056e-10  0.9999396473343  1.183903667831e-10  1900.055708261      
Optimization terminated successfully.
         Current fu

-- 2020-11-09 11:21:23 - muse.mca - WARNING
Check growth constraints for wind.



Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 647.4402248427      
0.1042702416317     0.1042702416317     0.1042702416317     0.9044651903652  0.1042702416317     197.7436068274      
0.04330783922975    0.04330783922975    0.04330783922975    0.6213338928986  0.04330783922975    554.8458695929      
0.02029339907042    0.02029339907042    0.02029339907042    0.5546164251713  0.02029339907042    887.503418141       
0.002059630009005   0.002059630008646   0.002059630008646   0.9123945707354  0.002059630008855   1773.497308596      
1.96164830749e-06   1.961648306624e-06  1.961648306623e-06  0.9995475315428  1.96164830693e-06   1899.795955905      
1.183904657193e-10  1.183907029537e-10  1.183907180056e-10  0.9999396473343  1.183903667831e-10  1900.055708261      
Optimization terminated successfully.
         Current fu

-- 2020-11-09 11:21:29 - muse.mca - WARNING
Check growth constraints for wind.



We can now check that the simulation has created the files that we expect. We also check that our "Hello, world!" message has printed:

In [5]:
all_txt_files = sorted((Path() / "Results").glob("Residential*.txt"))
assert "Hello world!" in all_txt_files[0].read_text()
all_txt_files

[PosixPath('Results/ResidentialConsumption_Zero2020.txt'),
 PosixPath('Results/ResidentialConsumption_Zero2025.txt'),
 PosixPath('Results/ResidentialConsumption_Zero2030.txt'),
 PosixPath('Results/ResidentialConsumption_Zero2035.txt'),
 PosixPath('Results/ResidentialConsumption_Zero2040.txt'),
 PosixPath('Results/ResidentialConsumption_Zero2045.txt'),
 PosixPath('Results/ResidentialConsumption_Zero2050.txt')]

Our model output the files we were expecting and passed the `assert` statement, meaning that it could find the "Hello world!" messages in the outputs.

## Adding TOML parameters to the outputs

It would be useful if we could pass parameters from the TOML file to our new functions `consumption_zero` and `text_dump`. For example, in our previous iteration the consumption output was aggregating the data by `"timeslice"`, by hardcoding the variable. We can pass a parameter which could do this by setting the `sum_over` parameter to be `True`. In addition, we could change the message output by a new `text_dump` function.

Not all hooks are this flexible (for historical reasons, rather than any intrinsic difficulty). However, for outputs, we can do this as follows:

In [6]:
@register_output_quantity(overwrite=True)
def consumption_zero(
    market: Dataset,
    capacity: DataArray,
    technologies: Dataset,
    sum_over: Optional[List[Text]] = None,
    drop: Optional[List[Text]] = None,
    rounding: int = 4,
):
    """Current consumption."""
    result = (
        market_quantity(market.consumption, sum_over=sum_over, drop=drop)
        .rename("consumption")
        .to_dataframe()
        .round(rounding)
    )
    return result


@register_output_sink(name="txt", overwrite=True)
@sink_to_file(".txt")
def text_dump(
    data: Any,
    filename: Text,
    msg : Optional[Text] = "Hello, world!"
) -> None:
    from pathlib import Path
    Path(filename).write_text(f"{msg}\n\n{data}")

We simply added parameters as arguments to both of our functions: `consumption_zero` and `text_dump`.

Note: The overwrite argument allows us to overwrite previously defined registered functions. This is useful in a notebook such as this. But it should not be used in general. If overwrite were false, then the code would issue a warning and it would leave the TOML to refer to the original functions at the beginning of the notebook. This is useful when using custom modules.

Now we can modify the output section to take additional arguments:

    [[sectors.commercial.outputs]]
    quantity.name = "consumption_zero"
    quantity.sum_over = "timeslice"
    sink.name = "txt"
    sink.filename = "{cwd}/{default_output_dir}/{Sector}{Quantity}{year}{suffix}"
    sink.msg = "Hello, you!"
    sink.overwrite = True
    
Here, we still want to use the `consumption_zero` function and the `txt` sink. But we would like to change the message from "Hello world!" to "Hello you!" within the `TOML` file.
    
Now, both sink and quantity are dictionaries which can take any number of arguments. Previously, we were using a shorthand for convenience. Again, we create a new settings file, and run this with our new parameters, which interface with our new functions.

In [7]:
from pathlib import Path
from toml import load, dump
from muse import examples

model_path = examples.copy_model(overwrite=True)
settings = load(model_path / "settings.toml")
settings["sectors"]["residential"]["outputs"] = [
    {
        "quantity":{
            "name": "consumption_zero",
            "sum_over": "timeslice"
        },
        "sink":{
            "name": "txt",
            "filename": "{cwd}/{default_output_dir}/{Sector}{Quantity}{year}{suffix}",
            "msg": "Hello, you!",
            "overwrite": True,
        }
        
    }
]

dump(settings, (model_path / "modified_settings_2.toml").open("w"))
settings

{'time_framework': [2020, 2025, 2030, 2035, 2040, 2045, 2050],
 'foresight': 5,
 'regions': ['R1'],
 'interest_rate': 0.1,
 'interpolation_mode': 'Active',
 'log_level': 'info',
 'equilibrium_variable': 'demand',
 'maximum_iterations': 100,
 'tolerance': 0.1,
 'tolerance_unmet_demand': -0.1,
 'outputs': [{'quantity': 'prices',
   'sink': 'aggregate',
   'filename': '{cwd}/{default_output_dir}/MCA{Quantity}.csv'},
  {'quantity': 'capacity',
   'sink': 'aggregate',
   'filename': '{cwd}/{default_output_dir}/MCA{Quantity}.csv'}],
 'carbon_budget_control': {'budget': []},
 'global_input_files': {'projections': '{path}/input/Projections.csv',
  'global_commodities': '{path}/input/GlobalCommodities.csv'},
 'sectors': {'residential': {'type': 'default',
   'priority': 1,
   'dispatch_production': 'share',
   'technodata': '{path}/technodata/residential/Technodata.csv',
   'commodities_in': '{path}/technodata/residential/CommIn.csv',
   'commodities_out': '{path}/technodata/residential/CommOut

We then run the simulation again:

In [10]:
mca = MCA.factory(model_path / "modified_settings_2.toml")
mca.run();

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 148.9679256735      
0.2598249156018     0.2598249156018     0.2598249156018     0.7495200004432  0.2598249156018     120.5733849622      
0.02399956829695    0.02399956829695    0.02399956829695    0.9210391224498  0.02399956829695    4.780663765494      
0.0181364461758     0.0181364461758     0.0181364461758     0.2509588065043  0.0181364461758     7.107141691547      
0.01499350833129    0.01499350833129    0.01499350833129    0.1921973185437  0.01499350833129    70.77614035582      
0.004968295711366   0.004968295711367   0.004968295711366   0.6857131120066  0.004968295711366   164.7472224003      
0.0006443120819652  0.0006443120819642  0.000644312081964   0.8804718592549  0.0006443120819672  289.7109372802      
2.427431365313e-06  2.427431365276e-06  2.42743136527e-06

-- 2020-11-09 11:32:00 - muse.mca - WARNING
Check growth constraints for wind.



Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 359.2443189825      
0.2131118149695     0.2131118149695     0.2131118149695     0.7960343319409  0.2131118149695     262.2885392799      
0.05758094728718    0.05758094728718    0.05758094728718    0.7523513573813  0.05758094728718    13.16175379893      
0.01048349585899    0.01048349585899    0.01048349585899    0.8203940076989  0.01048349585899    9.036399448741      
0.009005598049604   0.009005598049604   0.009005598049604   0.1488155054953  0.009005598049604   19.40296370843      
0.002317604003518   0.002317604003518   0.002317604003518   0.8098673571979  0.002317604003518   226.5492459765      
0.0009784310820962  0.0009784310820963  0.0009784310820963  0.5991550275447  0.0009784310820971  289.4684048776      
0.0001326875071986  0.0001326875071986  0.000132687507198

-- 2020-11-09 11:32:10 - muse.mca - WARNING
Check growth constraints for wind.



Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 359.2443189825      
0.2131118149695     0.2131118149695     0.2131118149695     0.7960343319409  0.2131118149695     262.2885392799      
0.05758094728718    0.05758094728718    0.05758094728718    0.7523513573813  0.05758094728718    13.16175379893      
0.01048349585899    0.01048349585899    0.01048349585899    0.8203940076989  0.01048349585899    9.036399448741      
0.009005598049604   0.009005598049604   0.009005598049604   0.1488155054953  0.009005598049604   19.40296370843      
0.002317604003518   0.002317604003518   0.002317604003518   0.8098673571979  0.002317604003518   226.5492459765      
0.0009784310820962  0.0009784310820963  0.0009784310820963  0.5991550275447  0.0009784310820971  289.4684048776      
0.0001326875071986  0.0001326875071986  0.000132687507198

...

/Users/alexkell/Documents/SGI/1-examples/example_model/model/Results/muse/src/muse/investments.py:325: OptimizeWarning: Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.
  res = linprog(**adapter.kwargs, options=dict(disp=True))
/Users/alexkell/Documents/SGI/1-examples/example_model/model/Results/muse/src/muse/investments.py:325: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
  res = linprog(**adapter.kwargs, options=dict(disp=True))
/Users/alexkell/anaconda3/lib/python3.8/site-packages/scipy/optimize/_linprog_ip.py:116: LinAlgWarning: Ill-conditioned matrix (rcond=1.62544e-36): result may not be accurate.
  return sp.linalg.solve(M

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 647.4402248427      
0.1042702416317     0.1042702416317     0.1042702416317     0.9044651903652  0.1042702416317     197.7436068274      
0.04330783922975    0.04330783922975    0.04330783922975    0.6213338928986  0.04330783922975    554.8458695929      
0.02029339907042    0.02029339907042    0.02029339907042    0.5546164251713  0.02029339907042    887.503418141       
0.002059630009005   0.002059630008646   0.002059630008646   0.9123945707354  0.002059630008855   1773.497308596      
1.96164830749e-06   1.961648306624e-06  1.961648306623e-06  0.9995475315428  1.96164830693e-06   1899.795955905      
1.183904657193e-10  1.183907029537e-10  1.183907180056e-10  0.9999396473343  1.183903667831e-10  1900.055708261      
Optimization terminated successfully.
         Current fu

-- 2020-11-09 11:33:46 - muse.mca - WARNING
Check growth constraints for wind.



Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 647.4402248427      
0.1042702416317     0.1042702416317     0.1042702416317     0.9044651903652  0.1042702416317     197.7436068274      
0.04330783922975    0.04330783922975    0.04330783922975    0.6213338928986  0.04330783922975    554.8458695929      
0.02029339907042    0.02029339907042    0.02029339907042    0.5546164251713  0.02029339907042    887.503418141       
0.002059630009005   0.002059630008646   0.002059630008646   0.9123945707354  0.002059630008855   1773.497308596      
1.96164830749e-06   1.961648306624e-06  1.961648306623e-06  0.9995475315428  1.96164830693e-06   1899.795955905      
1.183904657193e-10  1.183907029537e-10  1.183907180056e-10  0.9999396473343  1.183903667831e-10  1900.055708261      
Optimization terminated successfully.
         Current fu

-- 2020-11-09 11:33:58 - muse.mca - WARNING
Check growth constraints for wind.



And we can check the parameters were used accordingly:

In [9]:
all_txt_files = sorted((Path() / "Results").glob("Residential*.txt"))
assert len(all_txt_files) == 7
assert "Hello, you!" in all_txt_files[0].read_text()
all_txt_files

[PosixPath('Results/ResidentialConsumption_Zero2020.txt'),
 PosixPath('Results/ResidentialConsumption_Zero2025.txt'),
 PosixPath('Results/ResidentialConsumption_Zero2030.txt'),
 PosixPath('Results/ResidentialConsumption_Zero2035.txt'),
 PosixPath('Results/ResidentialConsumption_Zero2040.txt'),
 PosixPath('Results/ResidentialConsumption_Zero2045.txt'),
 PosixPath('Results/ResidentialConsumption_Zero2050.txt')]

Again, we can see that the number of output files generated were as we expected and that our new message "Hello, you!" was found within these files. This means that our output and sink functions worked as expected.

## Next steps

In the next section we will output a technology filter, to stop agents from investing in a certain technology, and a new metric to combine multiple objectives.